The codes in this notebooks are snippets of code from the src folder

This file is used mainly as a overview reference

# Book1.py

## Initial FASTAPI application

A simple application to get all books

In [4]:
from fastapi import FastAPI

app = FastAPI()

BOOKS = {
    'book1':{'title':'Title1','Author':'Author1'},
    'book2':{'title':'Title2','Author':'Author2'},
    'book3':{'title':'Title3','Author':'Author3'},
    'book4':{'title':'Title4','Author':'Author4'},
    'book5':{'title':'Title5','Author':'Author5'},
}

@app.get("/")
async def get_all_books():
    return BOOKS


## Command to run the uvicorn server

reload is use during developemnt, application reload when src code is saved
- uvicorn books:app --reload

URL to access
- http://127.0.0.1:8000
- http://127.0.0.1:8000/docs
- http://127.0.0.1:8000/openapi.json

In [ ]:
@app.get("/books/mybook")
async def read_favourite_book():
    return {"title":"My favourite book"}

# path parameter API has to be under any API that follow the same path
# /books/mybook will not work if below /books/{param}
@app.get("/books/{book_title}")
async def read_book(book_title):
    return {"title":book_title}

@app.get("/books/{book_id}")
async def read_book_id(book_id:int):
    return {"book_id":book_id}

## Enumerating the list of items from a class

Dropdown list for the parameters in the SwaggerUI using Enum

In [ ]:
from enum import Enum

# Using Enum to create drop downlist for parameters
class DirectionName(str, Enum):
    north = "North"
    south = "South"
    east = "East"
    west = "West"

@app.get("/directions/{direction_name}")
async def get_direction(direction_name: DirectionName):
    if direction_name == DirectionName.north:
        return{"Direction":direction_name, "sub":"Up"}
    if direction_name == DirectionName.west:
        return{"Direction":direction_name, "sub":"Left"}
    if direction_name == DirectionName.east:
        return{"Direction":direction_name, "sub":"Right"}
    return{"Direction":direction_name, "sub":"Down"}

## FASTAPI Parameterized

book is skipped based off the values pass to parameters

In [ ]:
from typing import Optional

@app.get("/books/skipbook")
async def skip_books(skip_book: Optional[str] = None):
    if skip_books:
        new_books = BOOKS.copy()
        del new_books[skip_book]
        return new_books
    return BOOKS

## POST Method

In [ ]:
@app.post("/")
async def create_book(title, author):
    current_book_id = 0
    if len(BOOKS) > 0:
        for book in BOOKS:
            x = int(book.split("_")[-1])
            if x > current_book_id:
                current_book_id = x
    BOOKS[f'book_{current_book_id+1}'] = {'title':title,'author':author}
    return BOOKS[f'book_{current_book_id+1}']

## PUT Method

In [ ]:
@app.put("/{book_name}")
async def update_book(book_name:str, title:str, author:str):
    book_information = {'title':title, 'author':author}
    BOOKS[book_name] = book_information
    return book_information

## Delete Method

In [ ]:
@app.delete("/{book_name}")
async def delete_book(book_name):
    del BOOKS[book_name]
    return f'{book_name} deleted'

# Book2.py

- pydantic 
    - Data validation and settings management using python type annotations.
    - pydantic enforces type hints at runtime, and provides user friendly errors when data is invalid.

- UUID
    - A universally unique identifier (UUID) is a 128-bit label used for information in computer systems

## Initial Application

In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from uuid import UUID

app = FastAPI()

# Inherit BaseModel to enforce type hinting
class Book(BaseModel):
    id:UUID
    title:str
    author:str
    description:str
    rating:int

BOOKS = []

@app.get("/")
async def read_all_books():
    return BOOKS

## POST Method

Request body is required

curl -X 'POST' \
  'http://127.0.0.1:8000/' \
  -H 'accept: application/json' \
  -H 'Content-Type: application/json' \
  -d '{
  "id": "3fa85f64-5717-4562-b3fc-2c963f66afa6",
  "title": "string",
  "author": "string",
  "description": "string",
  "rating": 0
}'

## Data Validation

- Field allow for data validation and Optional allows for ommision of the parameter

In [ ]:
from typing import Optional
from pydantic import BaseModel, Field

app = FastAPI()

# Uses BaseModel to enforce type hinting
class Book(BaseModel):
    id:UUID
    title:str = Field(min_length=1) # data validation 
    author:str = Field(min_length=1, max_length=100)
    description:Optional[str] = Field(
        title="Description of book",
        max_length=100,
        min_length=1)
    rating:int = Field(gt=-1,lt=101)

## Creating Schema Exampels

- inner class Config

In [ ]:
class Book(BaseModel):
    id:UUID
    title:str = Field(min_length=1) # data validation 
    author:str = Field(min_length=1, max_length=100)
    description:Optional[str] = Field(
        title="Description of book",
        max_length=100,
        min_length=1)
    rating:int = Field(gt=-1,lt=101)

    class Config:
        schema_extra = {
            "example":{
                "id":"94b47336-8ff8-4efe-bf89-981832ae6555",
                'title':"Title_1",
                "author":"Author_1",
                "description":"A very Nice Book"
            }
        }